In [1]:
import uuid
import os
import asyncio
from pprint import pp
from dotenv import load_dotenv
from tempfile import mkdtemp

from jobs import Worker
from output_data import run_sbml_amici, run_sbml_copasi, run_sbml_tellurium
from shared import MongoDbConnector, save_uploaded_file, upload_blob

load_dotenv('../assets/.env_dev')

MONGO_URI = os.getenv('MONGO_URI')
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
BUCKET_NAME = os.getenv('BUCKET_NAME')
DB_NAME = os.getenv('DB_NAME')

db_connector = MongoDbConnector(connection_uri=MONGO_URI, database_id=DB_NAME, connector_id="test_worker")

In [2]:
sbml_fp = "../model-examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml"

os.path.exists(sbml_fp)

True

In [3]:
params = {
    'sbml_fp': sbml_fp,
    'dur': 10,
    'n_steps': 100
}

amici_results = run_sbml_amici(**params)
copasi_results = run_sbml_copasi(**params)
tellurium_results = run_sbml_tellurium(**params)

2024-07-28 00:51:33.241 - amici.sbml_import - INFO - Finished importing SBML                         (2.47E-02s)
2024-07-28 00:51:33.266 - amici.sbml_import - INFO - Finished processing SBML observables            (2.41E-02s)
2024-07-28 00:51:33.269 - amici.sbml_import - INFO - Finished processing SBML event observables      (7.92E-07s)
2024-07-28 00:51:33.290 - amici.de_model - INFO - Finished computing xdot                            (1.65E-03s)
2024-07-28 00:51:33.293 - amici.de_model - INFO - Finished computing x0                              (1.30E-03s)
2024-07-28 00:51:33.304 - amici.de_model - INFO - Finished computing w                               (8.79E-03s)
2024-07-28 00:51:33.769 - amici.de_export - INFO - Finished generating cpp code                      (4.61E-01s)
2024-07-28 00:51:43.193 - amici.de_export - INFO - Finished compiling cpp code                       (9.42E+00s)


In [4]:
amici_results['floating_species']['cI mRNA']

array([0.        , 2.85494179, 5.00435142, 6.34482551, 7.09921386,
       7.4921065 , 7.66922871, 7.71593383, 7.68184878, 7.59649974,
       7.47803362, 7.33799025, 7.18394851, 7.02103276, 6.85279492,
       6.68175009, 6.5097088 , 6.33799227, 6.16757364, 5.99917395,
       5.83332823, 5.67043231, 5.51077647, 5.35457007, 5.20195993,
       5.05304386, 4.90788136, 4.76650165, 4.62890985, 4.49509208,
       4.36501922, 4.23865018, 4.11593432, 3.99681352, 3.88122379,
       3.76909669, 3.66036039, 3.55494054, 3.45276113, 3.35374495,
       3.25781429, 3.16489118, 3.07489784, 2.98775699, 2.903392  ,
       2.82172718, 2.74268789, 2.66620063, 2.59219321, 2.52059483,
       2.45133613, 2.38434922, 2.31956773, 2.25692685, 2.19636332,
       2.13781546, 2.08122316, 2.02652787, 1.97367259, 1.92260189,
       1.87326183, 1.82559999, 1.77956545, 1.73510871, 1.69218173,
       1.65073786, 1.61073183, 1.57211971, 1.53485889, 1.49890805,
       1.46422712, 1.43077725, 1.3985208 , 1.36742129, 1.33744

In [5]:
copasi_results['floating_species']['cI mRNA']

array([0.        , 2.82922732, 4.96978351, 6.31444297, 7.07722048,
       7.47851419, 7.66292437, 7.71573589, 7.68676691, 7.60575027,
       7.49100323, 7.35419088, 7.20298268, 7.04257201, 6.87656122,
       6.70750319, 6.53723749, 6.36710935, 6.19811155, 6.03098086,
       5.8662657 , 5.70437345, 5.54560395, 5.39017637, 5.2382452 ,
       5.08991547, 4.94525322, 4.80429363, 4.66704732, 4.53350537,
       4.40364315, 4.27742451, 4.15480254, 4.0357231 , 3.92012573,
       3.80794532, 3.69911324, 3.59355805, 3.49120657, 3.39198422,
       3.29581563, 3.20262525, 3.11233769, 3.02487753, 2.94017026,
       2.8581419 , 2.77871963, 2.70183156, 2.62740709, 2.55537687,
       2.48567288, 2.41822857, 2.35297876, 2.2898598 , 2.22880948,
       2.16976713, 2.1126737 , 2.05747146, 2.00410425, 1.95251747,
       1.90265788, 1.85447376, 1.80791484, 1.76293221, 1.7194784 ,
       1.6775073 , 1.6369741 , 1.59783534, 1.56004882, 1.52357362,
       1.48837001, 1.4543995 , 1.42162472, 1.39000948, 1.35951

In [6]:
tellurium_results['floating_species']['cI mRNA']

[0.         2.85494357 5.00434303 6.34481917 7.09922379 7.49212689
 7.66925107 7.71595069 7.68186175 7.5965112  7.47804357 7.33799329
 7.18394749 7.02102978 6.85279058 6.68174496 6.50970047 6.33797914
 6.16755521 5.99915367 5.8333079  5.67041131 5.51075506 5.35454906
 5.20193904 5.05302266 4.90785781 4.76647433 4.62887975 4.49506152
 4.36498963 4.23862143 4.11590571 3.99678481 3.88119541 3.76906922
 3.66033401 3.55491534 3.45273694 3.35372165 3.25779171 3.16486919
 3.07487643 2.98773618 2.90337178 2.82170763 2.74266896 2.66618236
 2.59217557 2.52057783 2.45131973 2.3843334  2.3195525  2.25691218
 2.19634923 2.13780193 2.08121015 2.02651531 1.97366048 1.92259013
 1.87325031 1.82558881 1.77955449 1.73509795 1.69217132 1.65072772
 1.61072201 1.57211029 1.53484987 1.49889945 1.46421894 1.4307695
 1.39851345 1.36741431 1.33743674 1.30854649 1.28071039 1.25389632
 1.22807318 1.20321085 1.17928018 1.15625293 1.13410181 1.11280037
 1.09232301 1.07264502 1.05374243 1.03559207 1.01817157 1.00145